In [1]:
import sys
import os
from code import InteractiveConsole

class SyntacticAnalyzer:
    def __init__(self):
        self.console = InteractiveConsole()
        self.instructions = []
        self.indentation_level = 0  # Track indentation for multi-line statements

    def is_complete(self, line):
        # Check if the line ends with a colon (for loops, conditionals)
        if line.strip().endswith(':'):
            self.indentation_level += 1
            return False
        elif self.indentation_level > 0:
            # Check for indentation level decrease (indicating end of block)
            if not line.startswith(' ' * 4):  # Assuming 4 spaces for indentation
                self.indentation_level -= 1
                return True
            return False
        return True

    def execute_line(self, line):
        try:
            if self.console.push(line):
                # If it's an incomplete statement, buffer it
                self.instructions.append(line)
                print("\t", end='')  # Indicate waiting for more input
            else:
                # Complete statement; execute buffered instructions
                self.instructions.append(line)
                full_code = ''.join(self.instructions)
                exec(full_code)  # Execute the full code block
                self.instructions.clear()  # Clear buffer after execution
                print("Executed successfully.")
        except Exception as e:
            print(f'Error: {e}')
            self.instructions.clear()  # Clear buffer on error

def main():
    analyzer = SyntacticAnalyzer()
    print('Toy test language interpreter')

    while True:
        try:
            line = sys.stdin.readline()
            if len(line) == 0:  # End of input
                sys.exit(0)

            analyzer.execute_line(line)

        except KeyboardInterrupt:
            print("\nExiting...")
            break

if __name__ == "__main__":
    main()


Toy test language interpreter


: 

In [ ]:
import json

pinmap=[]
for pinno, pin in enumerate(['IOCLK2','IODATA2','IODATA0','IODATA1','VDD','RESETB','VMID','IOCLK0','PVDD','PVDD','ADDR','GND','PGND','OUTP','OUTN','IOCLK1']):
    if pin in ['VDD','RESETB']:
        value=1.8
        unit='V'
    elif pin == 'PVDD':
        value=2.3
        unit='V'
    else:
        value=''
        unit=''

    pinmap.append(
        {
            'pinno':pinno+1,
            'pinname':pin,
            'value':value,
            'unit':unit
        }
    )
    # print(f'{pin}="{pin}"')
with open('aerodyne_pinmap.json','w')as file:
    json.dump(pinmap,file)
    

IOCLK2="IOCLK2"
IODATA2="IODATA2"
IODATA0="IODATA0"
IODATA1="IODATA1"
VDD="VDD"
RESETB="RESETB"
VMID="VMID"
IOCLK0="IOCLK0"
PVDD="PVDD"
PVDD="PVDD"
ADDR="ADDR"
GND="GND"
PGND="PGND"
OUTP="OUTP"
OUTN="OUTN"
IOCLK1="IOCLK1"


In [36]:
IOCLK2="IOCLK2"
IODATA2="IODATA2"
IODATA0="IODATA0"
IODATA1="IODATA1"
VDD="VDD"
RESETB="RESETB"
VMID="VMID"
IOCLK0="IOCLK0"
PVDD="PVDD"
PVDD="PVDD"
ADDR="ADDR"
GND="GND"
PGND="PGND"
OUTP="OUTP"
OUTN="OUTN"
IOCLK1="IOCLK1"

In [22]:
from dfttools import *
from time import sleep
import random

Test_Name = 'UVLO_H2L_Trim'
print(f'............ {Test_Name} ........')

'''
Set PVDD=2.25V (desired L2H UVLO threshold), 
bring UVLO comparator output to pin IODATA0 
through digital test mux and 
sweep the UVLO 0.4V reference voltage 
trimming code until the comparator toggles 
'''

HL_Th = 2.25  # 2.25V threshold
error_spread = 2.5e-3 # 2.5mv error spread
code_width = 4  # 4 bits trimming code
min_error = float('inf')
optimal_code = None
optimal_measured_value = None
force_voltage_low_limit = 0  # minimum voltage limit
force_voltage_high_limit = 3.7  # minimum voltage limit

for Code in range(2**code_width):
    # TODO: Write trimming code to device via I2C
    # Example (uncomment and fill device_address and field_info):
    # I2C_WRITE(device_address=0xXX, field_info=field_info, write_value=hex(Code))

    force_voltage = force_voltage_high_limit
    trigger = False

    while True:
        uvlo_forced_voltage = VFORCE(signal='A5V', reference='GND1-', value=force_voltage)
        # Add noise simulation
        # remove in the real test
        uvlo_forced_voltage += random.uniform(-error_spread, error_spread)

        # Check trigger condition
        trigger = VTRIG_HL(signal='PVDD', reference='GND1-', threshold=HL_Th, expected_value=force_voltage)

        if trigger:
            break
        elif uvlo_forced_voltage <= force_voltage_low_limit:
            print(f'..... Voltage min limit {force_voltage_low_limit}V crossed ........')
            break

        force_voltage -= 0.01  # decrease voltage by 10mV
        sleep(0.01)  # 10 ms delay

    error = abs(uvlo_forced_voltage - HL_Th)/abs(HL_Th) * 100
    if error < min_error:
        min_error = error
        optimal_code = hex(Code)
        optimal_measured_value = uvlo_forced_voltage

# Final check and reporting
if force_voltage_low_limit < optimal_measured_value < force_voltage_high_limit:
    print(f'............ Trim_Bg Test Passed ........')
    # TODO: Burn optimal trimming code to OTP via I2C
    # I2C_WRITE(device_address=0xXX, field_info=field_info, write_value=optimal_code)
else:
    print(f'............ Trim_Bg Test Failed ........')

print(f"Optimal Code: {optimal_code}")
print(f"Optimal Measured Value: {optimal_measured_value:.4f} V (Target: {HL_Th} V)")
print(f"Minimum Error: {min_error:.6f} %")














............ UVLO_H2L_Trim ........
............ Trim_Bg Test Passed ........
Optimal Code: 0x8
Optimal Measured Value: 2.2425 V (Target: 2.25 V)
Minimum Error: 0.334882 %


In [31]:
DEVADRR = 0x68

In [57]:
from dfttools import *
from time import sleep
import random

Test_Name = 'BG_1V2_Trim'
from Procedures import Startup
print(f'............ {Test_Name} ........')
'''
enable ref_bg, # designer needs identify
otp_ds_ref_bg_ptat_trm<2:0>=0d, 
otp_ds_ref_bg_trm_0v6<3:0>=0d, 
otp_ds_ref_bg_trm_0v9<3:0>=0d, 
otp_ds_ref_bg_trm_1v2<3:0>=0d; 
'''
# I2C_WRITE(device_address=DEVADRR,field_info=bg_ptat,value=0x0)
# I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v6,value=0x0)
# I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v9,value=0x0)
# I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_1v2,value=0x0)
# Bring out to the analog test point (rfu_ds_ref_test_en_vddd=4d) the 1.2V reference output voltage. Target voltage is 1.2V; 
# I2C_WRITE(device_address=DEVADRR,field_info=,value=)

'''
Bring 1.2V bandgap reference voltage to IODATA1 through the analog test mux and trim it to the closest value to 1.2V
'''

# Initial value
percentage = 0.2
typical_value = 1.2
low_value = typical_value - typical_value*percentage
high_value = typical_value + typical_value*percentage

# Step size
step_size = 0.01 # 10mV

# Number of steps width of the field / bits
num_steps = 2**4  # 4-bit

# Standard deviation for white noise
noise_std_dev = 0.025

# Initialize minimum error and optimal code
min_error = float('inf')
optimal_code = None
optimal_measured_value = None

for i in range(num_steps):
    # sweep trimg code
    # I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_1v2,value=hex(i))
    # Generate monotonic values with step size
    expected_values = low_value + i * step_size 
    # Add white noise to each value
    noisy_values = expected_values + random.uniform(-noise_std_dev,noise_std_dev)
    # Pass the noisy value as the expected measurement values
    # SaveMeas__Voltage__hwmute__GND
    measured_value = VMEASURE(signal=IODATA1, reference=GND, expected_value=noisy_values)
    error = abs(measured_value - typical_value)
    if error < min_error:
        min_error = error
        optimal_code = hex(i)
        optimal_measured_value = measured_value
    sleep(0.1)
# Check for limits
if low_value < optimal_measured_value < high_value:
    print(f'............ Trim_Bg Test Passed ........')
    # write the optimized code if the trim passed
    # I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_1v2,value=hex(i))
else:
    print(f'............ Trim_Bg Test Failed ........')
    # if the trimh failed program detult zero
    # I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_1v2,value=0)
print(f"Optimal Code: {optimal_code}")
print(f"Optimal measured value : {optimal_measured_value}")
print(f"Minimum Error: {min_error}")


............ BG_1V2_Trim ........
............ Trim_Bg Test Passed ........
Optimal Code: 0xe
Optimal measured value : 1.1112979162255046
Minimum Error: 0.0887020837744954


In [60]:
from dfttools import *
from time import sleep
import random

Test_Name = 'BG_0v9_Trim'
from Procedures import Startup
print(f'............ {Test_Name} ........')
'''
enable ref_bg, # designer needs identify
otp_ds_ref_bg_ptat_trm<2:0>=0d, 
otp_ds_ref_bg_trm_0v6<3:0>=0d, 
otp_ds_ref_bg_trm_0v9<3:0>=0d, 
otp_ds_ref_bg_trm_1v2<3:0>=0d; 
'''
# I2C_WRITE(device_address=DEVADRR,field_info=bg_ptat,value=0x0)
# I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v6,value=0x0)
# I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v9,value=0x0)
# I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_1v2,value=0x0)
# Bring out to the analog test point (rfu_ds_ref_test_en_vddd=5d) the 1.2V reference output voltage. Target voltage is 1.2V; 
# I2C_WRITE(device_address=DEVADRR,field_info=,value=)

'''
Bring 1.2V bandgap reference voltage to IODATA1 through the analog test mux and trim it to the closest value to 1.2V
'''

# Initial value
percentage = 0.1 # 10% difference
typical_value = 0.9
low_value = typical_value - typical_value*percentage
high_value = typical_value + typical_value*percentage

# Step size
step_size = 0.01 # 10mV

# Number of steps width of the field / bits
num_steps = 2**4  # 4-bit

# Standard deviation for white noise
noise_std_dev = 0.025

# Initialize minimum error and optimal code
min_error = float('inf')
optimal_code = None
optimal_measured_value = None

for i in range(num_steps):
    # sweep trimg code
    # I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v9,value=hex(i))
    # Generate monotonic values with step size
    expected_values = low_value + i * step_size 
    # Add white noise to each value
    noisy_values = expected_values + random.uniform(-noise_std_dev,noise_std_dev)
    # Pass the noisy value as the expected measurement values
    measured_value = VMEASURE(signal=IODATA1, reference=GND, expected_value=noisy_values)
    error = abs(measured_value - typical_value)/abs(typical_value) *100
    if error < min_error:
        min_error = error
        optimal_code = hex(i)
        optimal_measured_value = measured_value
    sleep(0.1)
# Check for limits
if low_value < optimal_measured_value < high_value:
    print(f'............ Trim_Bg Test Passed ........')
    # write the optimized code if the trim passed
    # I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v9,value=hex(i))
else:
    print(f'............ Trim_Bg Test Failed ........')
    # if the trimh failed program detult zero
    # I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v9,value=0)
print(f"Optimal Code: {optimal_code}")
print(f"Optimal measured value : {optimal_measured_value}V")
print(f"Minimum Error: {min_error}%")


............ BG_0v9_Trim ........
............ Trim_Bg Test Passed ........
Optimal Code: 0x9
Optimal measured value : 0.8941481349518956V
Minimum Error: 0.6502072275671579%


In [62]:
from dfttools import *
from time import sleep
import random

Test_Name = 'BG_0v6_Trim'
from Procedures import Startup
print(f'............ {Test_Name} ........')
'''
enable ref_bg, # designer needs identify
otp_ds_ref_bg_ptat_trm<2:0>=0d, 
otp_ds_ref_bg_trm_0v6<3:0>=0d, 
otp_ds_ref_bg_trm_0v9<3:0>=0d, 
otp_ds_ref_bg_trm_1v2<3:0>=0d; 
'''
# I2C_WRITE(device_address=DEVADRR,field_info=bg_ptat,value=0x0)
# I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v6,value=0x0)
# I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v9,value=0x0)
# I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_1v2,value=0x0)
# Bring out to the analog test point (rfu_ds_ref_test_en_vddd=6d) the 0.6V reference output voltage. Target voltage is 0.6V; 
# I2C_WRITE(device_address=DEVADRR,field_info=,value=)

'''
Bring 0.6V bandgap reference voltage to IODATA1 through the analog test mux and trim it to the closest value to 0.6V
'''

# Initial value
percentage = 0.1 # 10% difference
typical_value = 0.6
low_value = typical_value - typical_value*percentage
high_value = typical_value + typical_value*percentage

# Step size
step_size = 0.01 # 10mV

# Number of steps width of the field / bits
num_steps = 2**4  # 4-bit

# Standard deviation for white noise
noise_std_dev = 0.025

# Initialize minimum error and optimal code
min_error = float('inf')
optimal_code = None
optimal_measured_value = None

for i in range(num_steps):
    # sweep trimg code
    # I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v6,value=hex(i))
    # Generate monotonic values with step size
    expected_values = low_value + i * step_size 
    # Add white noise to each value
    noisy_values = expected_values + random.uniform(-noise_std_dev,noise_std_dev)
    # Pass the noisy value as the expected measurement values
    measured_value = VMEASURE(signal=IODATA1, reference=GND, expected_value=noisy_values)
    error = abs(measured_value - typical_value)/abs(typical_value) *100
    if error < min_error:
        min_error = error
        optimal_code = hex(i)
        optimal_measured_value = measured_value
    sleep(0.1)
# Check for limits
if low_value < optimal_measured_value < high_value:
    print(f'............ {Test_Name} Passed ........')
    # write the optimized code if the trim passed
    # I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v6,value=hex(optimal))
else:
    print(f'............ {Test_Name} Failed ........')
    # if the trimh failed program detult zero
    # I2C_WRITE(device_address=DEVADRR,field_info=bg_trm_0v6,value=0)
print(f"Optimal Code: {optimal_code}")
print(f"Optimal measured value : {optimal_measured_value}V, Target vlaue : {typical_value}V")
print(f"Minimum Error: {min_error}%")


............ BG_0v6_Trim ........
............ BG_0v6_Trim Passed ........
Optimal Code: 0x7
Optimal measured value : 0.5899144242089125V, Target vlaue : 0.6V
Minimum Error: 1.6809292985145812%
